#SocialAL Model

#Fit single alpha model to data - multiple subjects

KLS 2.17.20
Project info: https://osf.io/b48n2/

Model modified from : Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

In [1]:
import sys
print(sys.version)
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, alpha):
    Prob = Prob + alpha*(recip-Prob)
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    alpha = params[0]
    beta = params[1]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    totalLLH = 0 
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        # use the probability of the selection (choice-probability) to update log likelihood
        if choice != 0:
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
            #print(cprob)

            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], alpha)
            ev[trustee] = update_value(prob[trustee])
        
    return totalLLH

def model_fit_once(data):
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        params = [alpha, beta]
        
        #results = minimize(get_likelihood_action, 
                           #params, args =(data), method='BFGS', options = {'maxiter': 10000, 'disp': False})
        results = minimize(get_likelihood_action, 
                       params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 100)], 
                       options = {'maxiter': 10000, 'disp': False})
        return results

def model_fit(data):
    
    tries = 10 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        alpha=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        params = [alpha, beta]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (1e-10, 20)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
            
    return bestFit

Load and Clean Data

In [3]:
files = os.listdir('../data/modeling')

In [4]:
def load_and_clean(file):
    path = os.path.join('../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    return(data)

Fit Model

In [5]:
print(getcontext())
params = [model_fit(load_and_clean(file)) for file in files]

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


In [6]:
params

[      fun: 47.03008543999501
  hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
       jac: array([ 0.008162  , -0.00165414])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 243
       nit: 47
    status: 0
   success: True
         x: array([3.09914643e-03, 2.00000000e+01]),
       fun: 51.64479495954017
  hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
       jac: array([-2.13162821e-06, -4.26325641e-06])
   message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 51
       nit: 14
    status: 0
   success: True
         x: array([0.21207539, 0.33599046]),
       fun: 60.99695188927512
  hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
       jac: array([1.42108547e-06, 0.00000000e+00])
   message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
      nfev: 6
       nit: 1
    status: 0
   success: True
         x: array([0.e+00, 1.e-10]),
       fun: 62.383246250395004
  hess_inv: <2x2 LbfgsInvHessProduct with dtype=

In [7]:
params = pd.DataFrame(params)
params['id'] = [file[:-4] for file in files]
params['alpha'] = [params['x'][y][0] for y in range(len(params))]
params['beta'] = [params['x'][y][1] for y in range(len(params))]
params['-LLH'] = [params['fun'][y] for y in range(len(params))]
params = params[['id', 'alpha', 'beta', '-LLH']]
print(params)   

          id     alpha          beta       -LLH
0   sub-2013  0.003099  2.000000e+01  47.030085
1   sub-2007  0.212075  3.359905e-01  51.644795
2   sub-2006  0.000000  1.000000e-10  60.996952
3   sub-2012  0.000000  1.000000e-10  62.383246
4   sub-2004  0.180936  1.098443e+00  35.077751
..       ...       ...           ...        ...
68  sub-2009  0.000000  1.000000e-10  62.383246
69  sub-2037  1.000000  2.120724e-01  50.237647
70  sub-2023  0.482881  6.157203e-01  34.813597
71  sub-2022  0.000000  1.000000e-10  60.996952
72  sub-2036  0.000000  1.000000e-10  62.383246

[73 rows x 4 columns]


In [8]:
# save parameters in text file
params.to_csv(path_or_buf = '../output/single_alpha_model_params.csv', index = False)